# 🧠 Análisis de Nota Económica – NLP en Python

## 📌 Introducción

Este notebook presenta un análisis de lenguaje natural (NLP) aplicado a una nota publicada en el diario *La Nación* el **29 de junio de 2025**, escrita por **José Del Río**. El artículo se enfoca en el estado de ánimo y las preocupaciones de distintos referentes empresariales frente a la coyuntura económica argentina.

A través del uso de herramientas de NLP en Python, exploraremos el contenido de la nota para extraer patrones, reducir el texto a sus componentes esenciales y medir su carga emocional o polaridad.

## 🎯 Objetivos

- Preprocesar el texto mediante tokenización, lematización y remoción de stopwords.
- Extraer tokens relevantes del artículo y visualizar palabras frecuentes.
- Realizar un análisis de sentimientos o polaridad del texto completo.
- Observar las distintas opiniones expresadas por 5 referentes clave:
  - **Javier Bolzico**, presidente de Adeba (sector bancario).
  - **Gustavo Weiss**, titular de Camarco (infraestructura).
  - **Issel Kiperszmid**, vicepresidente de la CEDU (desarrollos inmobiliarios).
  - **Martín Rapallini**, presidente de la Unión Industrial Argentina (industria nacional).
  - **Mario Grinman**, titular de la Cámara Argentina de Comercio (comercio y servicios).

Estos representantes, junto con la Bolsa y la Sociedad Rural Argentina, forman un termómetro de los sectores más relevantes del país.

## 🔗 Fuente del artículo

[Plan económico: ¿cuál es el verdadero riesgo que preocupa a los empresarios? (La Nación)](https://www.lanacion.com.ar/economia/plan-economico-cual-es-el-verdadero-riesgo-que-preocupa-a-los-empresarios-nid29062025/)

## 🧰 Librerías necesarias

Este proyecto utilizará las siguientes bibliotecas de Python:

- **spaCy**: Para tokenización y lematización en idioma español.
- **nltk**: Para eliminación de *stopwords*.
- **textblob** y **textblob-es**: Para análisis de sentimientos.
- **matplotlib**: Para visualización gráfica de los resultados.
- **seaborn**: Para visualizaciones estadísticas mejoradas.
- **pandas**: Para manipulación y análisis de datos tabulares.
- **numpy**: Para operaciones numéricas y estructuras de datos eficientes.
- **vaderSentiment** *(opcional)*: Análisis de polaridad si se decide traducir el texto a inglés.